In [4]:
import pandas as pd
import numpy as np
import glob

In [2]:
dict_dtypes = {
    'IsBeta': np.int8,                                                              
    'RtpStateBitfield': np.int8,                                                    
    'IsSxsPassiveMode': np.int8,                                                    
    'DefaultBrowsersIdentifier': np.int16,                                          
    'AVProductStatesIdentifier': np.int32,                                          
    'AVProductsInstalled': np.int8,                                                 
    'AVProductsEnabled': np.int8,                                                   
    'HasTpm': np.int8,                                                              
    'CountryIdentifier': np.int8,                                                   
    'CityIdentifier': np.int32,                                                     
    'OrganizationIdentifier': np.int8,
    'GeoNameIdentifier': np.int16,                                                  
    'LocaleEnglishNameIdentifier': np.int16,                                        
    'OsBuild': np.int16,                                                            
    'OsSuite': np.int16,                                                            
    'IsProtected': np.int8,
    'AutoSampleOptIn': np.int8,                                                     
    'SMode': np.int8,                                                               
    'IeVerIdentifier': np.int16,                                                    
    'Firewall': np.int8,                                                            
    'UacLuaenable': np.int32,                                                       
    'Census_OEMNameIdentifier': np.int16,                                           
    'Census_OEMModelIdentifier': np.int32,
    'Census_ProcessorCoreCount': np.int8,                                           
    'Census_ProcessorManufacturerIdentifier': np.int8,
    'Census_ProcessorModelIdentifier': np.int16,                                    
    'Census_PrimaryDiskTotalCapacity': np.int32,                                    
    'Census_SystemVolumeTotalCapacity': np.int32,                                   
    'Census_HasOpticalDiskDrive': np.int8,                                          
    'Census_TotalPhysicalRAM': np.int32,                                            
    'Census_InternalPrimaryDiagonalDisplaySizeInInches': np.float32,                   
    'Census_InternalPrimaryDisplayResolutionHorizontal': np.int16,                  
    'Census_InternalPrimaryDisplayResolutionVertical': np.int16,                    
    'Census_InternalBatteryNumberOfCharges': np.int32,                              
    'Census_OSBuildNumber': np.int16,
    'Census_OSBuildRevision': np.int16,                                             
    'Census_OSInstallLanguageIdentifier': np.int8,                                  
    'Census_OSUILocaleIdentifier': np.int8,                                         
    'Census_IsPortableOperatingSystem': np.int8,                                    
    'Census_IsFlightingInternal': np.int8,                                          
    'Census_IsFlightsDisabled': np.int8,
    'Census_ThresholdOptIn': np.int8,                                               
    'Census_FirmwareManufacturerIdentifier': np.int16,                              
    'Census_FirmwareVersionIdentifier': np.int32,                                   
    'Census_IsSecureBootEnabled': np.int8,                                          
    'Census_IsWIMBootEnabled': np.int8,                                             
    'Census_IsVirtualDevice': np.int8,                                              
    'Census_IsTouchEnabled': np.int8,
    'Census_IsPenCapable': np.int8,                                                 
    'Census_IsAlwaysOnAlwaysConnectedCapable': np.int8,                             
    'Wdft_IsGamer': np.int8,                                                        
    'Wdft_RegionIdentifier': np.int8,                                               
    'HasDetections': np.int8,
    'Census_InternalBatteryType_informed': np.int8,                                 
    'ProductName_index': np.int8,                                                   
    'Platform_index': np.int8,                                                      
    'Processor_index': np.int8,                                                     
    'OsPlatformSubRelease_index': np.int8,                                          
    'SkuEdition_index': np.int8,                                                    
    'PuaMode_index': np.int8,                                                       
    'SmartScreen_index': np.int8,                                                   
    'Census_MDC2FormFactor_index': np.int8,                                         
    'Census_DeviceFamily_index': np.int8,                                           
    'Census_ProcessorClass_index': np.int8,                                         
    'Census_PrimaryDiskTypeName_index': np.int8,                                    
    'Census_ChassisTypeName_index': np.int8,                                        
    'Census_PowerPlatformRoleName_index': np.int8,                                  
    'Census_InternalBatteryType_index': np.int8,                                    
    'Census_OSArchitecture_index': np.int8,                                         
    'Census_OSBranch_index': np.int8,                                               
    'Census_OSEdition_index': np.int8,                                              
    'Census_OSSkuName_index': np.int8,                                              
    'Census_OSInstallTypeName_index': np.int8,                                      
    'Census_OSWUAutoUpdateOptionsName_index': np.int8,                              
    'Census_GenuineStateName_index': np.int8,                                       
    'Census_ActivationChannel_index': np.int8,                                      
    'Census_FlightRing_index': np.int8,                                             
    'Census_OSVersion_0': np.int8,                                                  
    'Census_OSVersion_1': np.int8,                                                  
    'Census_OSVersion_2': np.int16,
    'Census_OSVersion_3': np.int16,                                                 
    'EngineVersion_2': np.int16,                                                    
    'EngineVersion_3': np.int8,                                                     
    'AppVersion_1': np.int8,                                                        
    'AppVersion_2': np.int16,                                                       
    'AppVersion_3': np.int16,                                                       
    'AvSigVersion_0': np.int8,                                                      
    'AvSigVersion_1': np.int16,                                                     
    'AvSigVersion_2': np.int16,                                                     
    'OsVer_0': np.int8,                                                             
    'OsVer_1': np.int8,                                                             
    'OsVer_2': np.int8,                                                             
    'OsVer_3': np.int8,                                                             
    'OsBuildLab_0': np.int16,                                                       
    'OsBuildLab_1': np.int16,                                                       
    'OsBuildLab_4_0': np.int32,                                                     
    'OsBuildLab_4_1': np.int16,                                                     
    'OsBuildLab_2_index': np.int8,                                                  
    'OsBuildLab_3_index': np.int8
}

In [3]:
path = '/home/errodringer/Escritorio/Microsoft/data/train_final_0'

In [5]:
allFiles = glob.glob(path + "/*.csv")

list_ = []
n = 0
for file_ in allFiles:
    df = pd.read_csv(file_, dtype=dict_dtypes)
    list_.append(df)
    print(n)
    n+=1

frame = pd.concat(list_, axis = 0, ignore_index = True)

In [7]:
frame.shape

(8921483, 100)

In [8]:
frame.memory_usage(deep=True).sum()/1024**2

2075.9982414245605

In [9]:
x = pd.read_csv('/home/errodringer/Escritorio/Microsoft/data/train_final_0/part-00000-3ebf38f3-2332-4f46-8b15-719fc40e5063-c000.csv',
                dtype=dict_dtypes)

In [10]:
x.head()

,MachineIdentifier,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,...,OsVer_0,OsVer_1,OsVer_2,OsVer_3,OsBuildLab_0,OsBuildLab_1,OsBuildLab_4_0,OsBuildLab_4_1,OsBuildLab_2_index,OsBuildLab_3_index
0,00001a18d69bb60bda9779408dcf02ac,0,7,0,0,46413,2,1,1,93,...,10,0,0,0,16299,431,180502,1908,0,2
1,00016c9d7670346f6d2f256c0091745a,0,7,0,0,53447,1,1,1,-55,...,10,0,0,0,17134,1,180410,1804,0,0
2,00017fa35ed739442dfc8787ef8d8182,0,7,0,0,53447,1,1,1,60,...,10,0,0,0,14393,0,160715,1616,0,4
3,000c754dde2a93af8883bfbfaa75ada7,0,7,0,0,62773,1,1,1,-85,...,10,0,0,0,15063,0,170317,1834,0,3
4,000e126b8eb014b70b556f8ed32a4a5d,0,7,0,0,53447,1,1,1,80,...,10,0,0,0,17134,1,180410,1804,0,0


In [20]:
x.memory_usage(deep=True).sum()/1024**2

10.38369369506836

In [21]:
10.3*200

2060.0